In [1]:
import os

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm

In [2]:
from sequential.dataset_seq import Sequence, SequenceLoader
from feature_extractor import feature_extractor

In [3]:
ann_dir = os.path.join(".", "mock data")
root_dir = os.path.join(".", "mock data", "actions")

In [4]:
ACTIVITIES = {
    "take leg": 0,
    "assemble leg": 1,
    "grab drill": 2,
    "use drill": 3,
    "drop drill": 4,
    "take screw driver": 5,
    "use screw driver": 6,
    "drop screw driver": 7
}

In [5]:
# read all sequences
seqs = []
for activity in os.listdir(root_dir):
    folders = [folder for folder in os.listdir(os.path.join(root_dir, activity))\
               if os.path.isdir(os.path.join(root_dir, activity, folder))]
    for folder in folders:
        path = os.path.join(root_dir, activity, folder)
        seq = Sequence(path, ACTIVITIES[activity], walk=True)
        seqs.append(seq)

In [6]:
seq_loader = SequenceLoader(seqs, feature_extractor)

In [7]:
# test data loading
# for feature_vectors, activity in dataloader:
#     print(feature_vectors.shape)
#     print(activity)
#     break

In [8]:
from sequential.model import LSTMActionClassifier

In [9]:
feature_vector_size = 1000
hidden_size = 512
batch_size = 8
num_frames = 3 # equals sequence length

dataloader = DataLoader(seq_loader, batch_size=batch_size, shuffle=True)
print(len(dataloader))

num_classes = 8
model = LSTMActionClassifier(feature_vector_size,
                             num_frames,
                             hidden_size,
                             num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

105


In [10]:
train_loss = []
for epoch in range(num_epochs):
    running_loss = 0
    # x = feature vectors
    # y = activity
    for i, (x, y) in tqdm(enumerate(dataloader), total=len(dataloader)):
        y = y.to(device)
        x = x.to(device).float() # edit: x is no of shape [8, 3, 1000]
        activity_oh = F.one_hot(y, num_classes).squeeze(0).float()
        optimizer.zero_grad()

        out = model(x)
        loss = criterion(out, activity_oh)
        
        loss.backward()
        optimizer.step()

        train_loss.append(loss.item())
        running_loss += loss.item()
    epoch_loss = running_loss / len(dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

  0%|          | 0/105 [00:00<?, ?it/s]c:\Users\Marc\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
100%|██████████| 105/105 [16:06<00:00,  9.20s/it]


Epoch [1/10], Loss: 2.0826


100%|██████████| 105/105 [15:42<00:00,  8.98s/it]


Epoch [2/10], Loss: 2.0730


100%|██████████| 105/105 [15:51<00:00,  9.06s/it]


...

Epoch [8/10], Loss: 2.0697


100%|██████████| 105/105 [15:29<00:00,  8.85s/it]


Epoch [9/10], Loss: 2.0687


100%|██████████| 105/105 [15:51<00:00,  9.06s/it]

Epoch [10/10], Loss: 2.0680


In [11]:
with torch.no_grad():
    model.eval()
    for _ in range(1):
        x, y = next(iter(dataloader))
        y = y.to(device)
        x = x.to(device).float()
        x_hats = model(x)
        for x_hat in x_hats:
            print(torch.argmax(x_hat))
        print(y)

tensor(6, device='cuda:0')
tensor(6, device='cuda:0')
tensor(6, device='cuda:0')
tensor(6, device='cuda:0')
tensor(6, device='cuda:0')
tensor(6, device='cuda:0')
tensor(6, device='cuda:0')
tensor(6, device='cuda:0')
tensor([0, 6, 1, 3, 4, 1, 6, 6], device='cuda:0')


In [12]:
torch.save(model.state_dict(), "seq_model_params.pth")